# Import Libraries

In [4]:
import os,shutil
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Create Directories for Small Data Set

In [5]:
#original_dataset_dir = '/Users/shruthiravi/Documents/Shruthi 9th grade/SkinLesionsSF18/cats_dogs/data_set/train'
base_dir = '/Users/shruthiravi/Documents/11th_grade/ushackathon_2'

In [6]:
os.mkdir(base_dir)

FileExistsError: [Errno 17] File exists: '/Users/shruthiravi/Documents/11th_grade/ushackathon_2'

In [16]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

FileExistsError: [Errno 17] File exists: '/Users/shruthiravi/Documents/11th_grade/ushackathon_2/train'

In [24]:
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

FileExistsError: [Errno 17] File exists: '/Users/shruthiravi/Documents/11th_grade/ushackathon_2/validation'

In [25]:
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

FileExistsError: [Errno 17] File exists: '/Users/shruthiravi/Documents/11th_grade/ushackathon_2/test'

In [26]:
train_lupus_dir = os.path.join(train_dir, 'lupus')
os.mkdir(train_lupus_dir)

In [27]:
train_exa_dir = os.path.join(train_dir, 'exa')
os.mkdir(train_exa_dir)

In [28]:
validation_lupus_dir = os.path.join(validation_dir, 'lupus')
os.mkdir(validation_lupus_dir)

In [29]:
validation_exa_dir = os.path.join(validation_dir, 'exa')
os.mkdir(validation_exa_dir)

In [30]:
test_lupus_dir = os.path.join(test_dir, 'lupus')
os.mkdir(test_lupus_dir)

In [31]:
test_exa_dir = os.path.join(test_dir, 'exa')
os.mkdir(test_exa_dir)

# Verify Image Counts

In [39]:
print('total training lupus images:', len(os.listdir(train_lupus_dir)))

total training lupus images: 50


In [40]:
print('total training exa images:', len(os.listdir(train_exa_dir)))

total training exa images: 91


In [41]:
print('total validation lupus images:', len(os.listdir(validation_lupus_dir)))

total validation lupus images: 15


In [42]:
print('total validation exa images:', len(os.listdir(validation_exa_dir)))

total validation exa images: 36


In [43]:
print('total test lupus images:', len(os.listdir(test_lupus_dir)))

total test lupus images: 30


In [44]:
print('total test exa images:', len(os.listdir(test_exa_dir)))

total test exa images: 55


# Data Preprocessing: Create Image Batch Generators

In [45]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [46]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [47]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

Found 139 images belonging to 2 classes.


In [48]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

Found 50 images belonging to 2 classes.


In [49]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


In [50]:
data_batch.shape

(20, 150, 150, 3)

In [51]:
labels_batch[1]

0.0

# Define the Model

In [52]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu',
                       input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

### Configuring model for training

In [54]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

# Fitting model using Batch Generator

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/20
100/100 [==============================] - 156s 2s/step - loss: 0.6241 - acc: 0.6630 - val_loss: 0.6372 - val_acc: 0.7036
Epoch 2/20
100/100 [==============================] - 256s 3s/step - loss: 0.4633 - acc: 0.7856 - val_loss: 0.7896 - val_acc: 0.6226
Epoch 3/20
100/100 [==============================] - 159s 2s/step - loss: 0.3170 - acc: 0.8726 - val_loss: 0.8484 - val_acc: 0.6810
Epoch 4/20
100/100 [==============================] - 154s 2s/step - loss: 0.1947 - acc: 0.9262 - val_loss: 1.0193 - val_acc: 0.7012
Epoch 5/20
100/100 [==============================] - 148s 1s/step - loss: 0.1109 - acc: 0.9707 - val_loss: 1.1975 - val_acc: 0.6810
Epoch 6/20
100/100 [==============================] - 332s 3s/step - loss: 0.0574 - acc: 0.9863 - val_loss: 1.3788 - val_acc: 0.6810
Epoch 7/20
100/100 [==============================] - 169s 2s/step - loss: 0.0428 - acc: 0.9899 - val_loss: 1.6024 - val_acc: 0.6595
Epoch 8/20
100/100 [==============================] - 170s 2s/step - 

In [ ]:
model.save('lyme_psy_1.h5')

In [56]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.xlabel("Threshold")
    plt.legend()
    plt.xlim([-70000, 70000])
    plt.ylim([0, 1])


def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-")  # b- means blue not dashed
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.xlim([0, 1])
    plt.ylim([0, 1])


def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')  # dashed diagonal
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.xlim([0, 1])
    plt.ylim([0, 1])

In [57]:
plot_roc_curve()

TypeError: plot_roc_curve() missing 2 required positional arguments: 'fpr' and 'tpr'

# Making a graph

In [32]:
import matplotlib.pyplot as plt

In [43]:
history.history

{'acc': [0.59581152002536819,
  0.73821989801853738,
  0.82984293193717273,
  0.85194805225768644,
  0.88759894012461238,
  0.9052355993168516,
  0.93193716856197539,
  0.9428571380578078,
  0.95445025466499533,
  0.96623376127961391,
  0.98153033923347899,
  0.98219895050787798,
  0.98638743318188249,
  0.98909090896705532,
  0.99267015719288931,
  0.99005235583370266,
  0.99633507828437851,
  0.99476439806179229,
  0.9942857136974087,
  0.99630606897902674],
 'loss': [0.65777834813008129,
  0.54699216221327562,
  0.42521151705250065,
  0.35032023928769224,
  0.29073174445248529,
  0.22943141892630392,
  0.17997835497542514,
  0.1469591499850541,
  0.1136235669876951,
  0.094936818327125791,
  0.060401536105438698,
  0.055505002920725507,
  0.042954180100890055,
  0.032712897025774436,
  0.025281624478000843,
  0.0281950870322,
  0.012506312035496228,
  0.019086492912729656,
  0.019336349907822389,
  0.015552094210586245],
 'val_acc': [0.71509009217088282,
  0.70608108047697993,
  0.7

In [ ]:
def plot_roc_curve

# Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
from keras.preprocessing import image

In [ ]:
fnames = [os.path.join(train_cats_dir, fname) for
         fname in os.listdir(train_cats_dir)]

In [ ]:
img_path = fnames[3]

In [ ]:
img = image.load_img(img_path, target_size=(150,150))

In [ ]:
x = image.img_to_array(img)

In [ ]:
x = x.reshape((1,) + x.shape)

In [ ]:
i = 0
for batch in datagen.flow(x,batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0 :
        break

In [ ]:
plt.show()

# Adding Dropout

In [47]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu',
                       input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [48]:
train_datagen = ImageDataGenerator (
    rescale=1./255,
    rotation_range=40,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=-.2,
    horizontal_flip=True,)

In [49]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [50]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

Found 325 images belonging to 2 classes.


In [51]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

Found 106 images belonging to 2 classes.


In [55]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/20
100/100 [==============================] - 161s 2s/step - loss: 0.6858 - acc: 0.5411 - val_loss: 0.6723 - val_acc: 0.5946
Epoch 2/20
100/100 [==============================] - 144s 1s/step - loss: 0.6499 - acc: 0.6250 - val_loss: 1.0759 - val_acc: 0.4291
Epoch 3/20
100/100 [==============================] - 149s 1s/step - loss: 0.6064 - acc: 0.6690 - val_loss: 0.7267 - val_acc: 0.5045
Epoch 4/20
100/100 [==============================] - 164s 2s/step - loss: 0.5589 - acc: 0.7275 - val_loss: 0.6584 - val_acc: 0.6667
Epoch 5/20
100/100 [==============================] - 153s 2s/step - loss: 0.5446 - acc: 0.7090 - val_loss: 0.5554 - val_acc: 0.7534
Epoch 6/20
100/100 [==============================] - 148s 1s/step - loss: 0.5013 - acc: 0.7535 - val_loss: 0.5598 - val_acc: 0.7860
Epoch 7/20
100/100 [==============================] - 182s 2s/step - loss: 0.4755 - acc: 0.7715 - val_loss: 0.4800 - val_acc: 0.7534
Epoch 8/20
100/100 [==============================] - 164s 2s/step - 

KeyboardInterrupt: 

In [2]:
model.save('cats_and_dogs_2.h5')

NameError: name 'model' is not defined

# Importing VGG16 Model

In [44]:
from keras.applications import VGG16

In [45]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150,150,3))

In [46]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________